<a href="https://colab.research.google.com/github/alenarth/ot-voos/blob/main/algoritmo_de_otimiza%C3%A7%C3%A3o_calendario_de_voos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Representação do problema


In [1]:
pessoas = [('Lisboa', 'LIS'),
           ('Madrid', 'MAD'),
           ('Paris', 'CDG'),
           ('Dublin', 'DUB'),
           ('Bruxelas', 'BRU'),
           ('Londres','LHR')]

In [3]:
destino = 'FCO'

In [4]:
voos = {('BRU', 'FCO'): ['15:44', '18:55', 382]}

In [15]:
voos = {}
for linha in open('/content/flights.txt'):
  #print(linha.split(','))
  origem, destino, saida, chegada, preco = linha.split(',')
  voos.setdefault((origem, destino), [])
  voos[(origem, destino)].append((saida, chegada, int(preco)))

In [16]:
agenda = [1,2, 3,2, 7,3, 6,3, 2,4, 5,3]
len(agenda)

12

In [17]:
def imprimir_voos(agenda):
  id_voo = -1
  total_preco = 0
  for i in range(len(agenda) // 2):
    nome = pessoas[i][0]
    origem = pessoas[i][1]
    id_voo += 1
    ida = voos[(origem, destino)][agenda[id_voo]]
    total_preco += ida[2]
    id_voo += 1
    volta = voos[(destino, origem)][agenda[id_voo]]
    total_preco += volta[2]
    print('%10s%10s %5s-%5s %3s %5s-%5s %3s' % (nome, origem, ida[0], ida[1], ida[2],
                                                volta[0], volta[1], volta[2]))
  print('Preço total: ', total_preco)

In [18]:
agenda = [1,0, 3,2, 7,1, 6,3, 2,4, 5,3]
imprimir_voos(agenda)

    Lisboa       LIS  7:39-10:24 219  6:19- 8:13 239
    Madrid       MAD 11:01-12:39 260  9:11-10:42 172
     Paris       CDG 17:07-20:04 291  8:23-11:07 143
    Dublin       DUB 15:27-17:18 151 10:33-12:03  74
  Bruxelas       BRU  9:08-12:12 364 12:20-16:34 500
   Londres       LHR 13:40-15:38 137 10:32-13:16 139
Preço total:  2689


## Fitness Function
O objetivo dessa fitness funtion é fazer uma leitura e nos retornar o preço das passagens.

In [22]:
def fitness_function(agenda):
  id_voo = -1
  total_preco = 0
  for i in range(len(agenda) // 2):
    origem = pessoas[i][1]
    id_voo += 1
    ida = voos[(origem, destino)][agenda[id_voo]]
    total_preco += ida[2]
    id_voo += 1
    volta = voos[(destino, origem)][agenda[id_voo]]
    total_preco += volta[2]

  return total_preco

In [23]:
agenda = [1,0, 3,2, 7,1, 6,3, 2,4, 5,3]
fitness_function(agenda)

2689

In [25]:
!git clone https://github.com/gkhayes/mlrose.git
!pip install mlrose/.

Cloning into 'mlrose'...
remote: Enumerating objects: 853, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 853 (delta 0), reused 4 (delta 0), pack-reused 849
Receiving objects: 100% (853/853), 414.35 KiB | 6.28 MiB/s, done.
Resolving deltas: 100% (551/551), done.
Processing ./mlrose
  Preparing metadata (setup.py) ... done
  Created wheel for mlrose: filename=mlrose-1.3.0-py3-none-any.whl size=27710 sha256=2972ccc7d11ea9f0585a6fa7eee5a81b9919914ac9e9d0f4352a43a737c0c124
  Stored in directory: /tmp/pip-ephem-wheel-cache-z5h9_zrv/wheels/50/ac/56/a248001dcf63344fbef992039dec2613cb4a7c56222c6f9a92
Successfully built mlrose


In [27]:
import six
import sys
sys.modules['sklearn.externals.six'] = six

In [28]:
import mlrose

In [32]:
fitness = mlrose.CustomFitness(fitness_function)

In [33]:
problema = mlrose.DiscreteOpt(length=12, fitness_fn=fitness,
                              maximize = False, max_val = 10)

## Hill climb

In [35]:
melhor_solucao, melhor_custo = mlrose.hill_climb(problema)
melhor_solucao, melhor_custo

(array([2, 6, 5, 7, 8, 1, 3, 6, 0, 2, 9, 1]), 1566.0)

In [36]:
imprimir_voos(melhor_solucao)

    Lisboa       LIS  9:15-12:03  99 15:07-17:21 129
    Madrid       MAD 14:22-16:32 126 17:06-20:00  95
     Paris       CDG 18:23-21:35 134  8:23-11:07 143
    Dublin       DUB 11:16-13:29  83 15:25-16:58  62
  Bruxelas       BRU  6:12-10:22 230  9:49-13:51 229
   Londres       LHR 20:30-23:11 114  8:19-11:16 122
Preço total:  1566


## Simulated annealing

In [48]:
melhor_solucao, melhor_custo = mlrose.simulated_annealing(problema,
                                                        #   schedule=mlrose.decay.GeomDecay(init_temp=10000), # ATUALIZAÇÃO FEV-2024
                                                          schedule=mlrose.GeomDecay(init_temp=10000), # ATUALIZAÇÃO FEV-2024
                                                          random_state=1)
melhor_solucao, melhor_custo

(array([6, 7, 4, 9, 8, 4, 1, 0, 0, 3, 5, 3]), 1820.0)

In [49]:
imprimir_voos(melhor_solucao)

    Lisboa       LIS 15:03-16:42 135 16:35-18:56 144
    Madrid       MAD 12:44-14:17 134 19:32-21:25 160
     Paris       CDG 18:23-21:35 134 12:37-15:05 170
    Dublin       DUB  8:04-10:11  95  6:39- 8:09  86
  Bruxelas       BRU  6:12-10:22 230 10:51-14:16 256
   Londres       LHR 13:40-15:38 137 10:32-13:16 139
Preço total:  1820


## Algoritmo genético


In [50]:
melhor_solucao, melhor_custo = mlrose.genetic_alg(problema, pop_size=500, mutation_prob=0.2,
                                                  random_state=1)
melhor_solucao, melhor_custo

(array([7, 3, 1, 2, 9, 0, 3, 8, 0, 3, 4, 1]), 1956.0)

In [51]:
imprimir_voos(melhor_solucao)

    Lisboa       LIS 16:51-19:09 147 11:07-13:24 171
    Madrid       MAD  8:25-10:34 157  9:11-10:42 172
     Paris       CDG 19:53-22:21 173  6:33- 9:14 172
    Dublin       DUB 11:16-13:29  83 18:24-20:49 124
  Bruxelas       BRU  6:12-10:22 230 10:51-14:16 256
   Londres       LHR 12:08-14:59 149  8:19-11:16 122
Preço total:  1956
